<a href="https://colab.research.google.com/github/sharathir/llmEngineering_CollabExercises/blob/main/Week3_Day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [ ]:
!pip install faster-whisper

In [ ]:
import os
import requests
from faster_whisper import WhisperModel
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import whisper
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/denver_extract.mp3"

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
import torch
torch.cuda.empty_cache()
print("Loading Whisper model...")
model = WhisperModel("base", device="cuda", compute_type="float16")
print("Transcribing audio...")
segments, info = model.transcribe(audio_filename, beam_size=5) # You can adjust beam_size
print(f"Detected language '{info.language}' with probability {info.language_probability:.2f}")
full_text = ""
print("\nTranscription:")
for segment in segments:
  print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
  full_text += segment.text + " "
print("\nFull Transcription:")
print(full_text.strip())

In [ ]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{full_text}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model1 = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map="auto", quantization_config=quant_config)
outputs = model1.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))